In [3]:
import pandas as pd

df = pd.read_excel('./Data/drone_data.xlsx')

print(df.head())

   Maximum Principal Elastic Strain (m/m)  Total Deformation (m)  \
0                            1.645777e-08           1.194049e-09   
1                            1.462148e-08           1.061343e-09   
2                            1.278519e-08           9.286820e-10   
3                            1.096267e-08           7.960099e-10   
4                            9.174577e-09           6.633378e-10   

   Equivalent Elastic Strain (m/m) Dimensions (mm)  
0                     2.171072e-08     400*410*100  
1                     1.929829e-08     400*410*100  
2                     1.688586e-08     400*410*100  
3                     1.447343e-08     400*410*100  
4                     1.206215e-08     400*410*100  


In [4]:
# Split the 'dimensions' column into three separate columns
df[['length', 'width', 'height']] = df['Dimensions (mm)'].str.split('*', expand=True)

# Convert the new columns to numeric values
df['length'] = pd.to_numeric(df['length'])
df['width'] = pd.to_numeric(df['width'])
df['height'] = pd.to_numeric(df['height'])

df.drop('Dimensions (mm)', axis=1, inplace=True)

df

,Maximum Principal Elastic Strain (m/m),Total Deformation (m),Equivalent Elastic Strain (m/m),length,width,height
0,1.645777e-08,1.194049e-09,2.171072e-08,400,410,100
1,1.462148e-08,1.061343e-09,1.929829e-08,400,410,100
2,1.278519e-08,9.286820e-10,1.688586e-08,400,410,100
3,1.096267e-08,7.960099e-10,1.447343e-08,400,410,100
4,9.174577e-09,6.633378e-10,1.206215e-08,400,410,100
5,7.294672e-09,5.306771e-10,9.649259e-09,400,410,100
6,5.476743e-09,3.980049e-10,7.236944e-09,400,410,100
7,3.647336e-09,2.653328e-10,4.824629e-09,400,410,100
8,1.788090e-09,1.326262e-10,2.412315e-09,400,410,100
9,2.144303e-09,1.557213e-10,2.831479e-09,400,410,100


In [5]:
import pandas as pd
import numpy as np

from skopt.space import Real, Integer
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.optimizers import Adam

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

In [6]:
def branin(X, Y):
    a = 1.0
    b = 5.1 / (4.0 * np.pi**2)
    c = 5.0 / np.pi
    r = 6.0
    s = 10.0
    t = 1.0 / (8.0 * np.pi)
    
    return a * (Y - b * X**2 + c * X - r)**2 + s * (1 - t) * np.cos(X) + s

def currin(X, Y):
    term1 = 1 - np.exp(-1 / (2 * Y))
    term2 = (2300 * X**3 + 1900 * X**2 + 2092 * X + 60) / (100 * X**3 + 500 * X**2 + 4 * X + 20)
    
    return term1 * term2

In [7]:
ran_err = lambda n, x: np.random.normal(0, x, n)

In [8]:
# number of initial simulations
sim_n = 10

# Random Seed
r = np.random.default_rng(seed=2)

In [9]:
# xy_range = [(-10, 10)]*2

# X = np.arange(*xy_range[0], ((xy_range[0][1] - xy_range[0][0]) / 100))
# Y = np.arange(*xy_range[1], ((xy_range[1][1] - xy_range[1][0]) / 100))
# X, Y = np.meshgrid(X, Y)

# # Generate the initial simulated experiments
# df = pd.DataFrame(r.uniform(xy_range[0][0], xy_range[0][1], size=(sim_n, 2)))
# df.columns = ["x", "y"]
# df["z1"] = obj_func(df['x'], df['y']) + ran_err(sim_n, 0.05)
# df["z2"] = obj_func2(df['x'], df['y']) + ran_err(sim_n, 0.05)

In [10]:
# Generate 10 random data points for two inputs (X, Y) within specified bounds
X = np.random.uniform(-5, 10, 10)
Y = np.random.uniform(0, 15, 10)

# Calculate the outputs using Branin and Currin functions
output1 = branin(X, Y)
output2 = currin(X, Y)

# Create a pandas DataFrame
df = pd.DataFrame({
    'x': X,
    'y': Y,
    'z1': output1,
    'z2': output2
})


In [11]:
df

,x,y,z1,z2
0,2.852613,13.280663,116.778420,0.430851
1,-4.055142,1.076220,186.439660,-31.148094
2,-0.436058,11.140952,38.261023,-0.284814
3,3.772406,5.381908,14.830217,1.123002
4,8.331399,2.066763,5.717275,3.433583
5,5.002212,3.383983,17.207953,1.895613
6,-3.460087,7.257013,34.480319,-2.868948
7,0.554289,14.079033,97.757962,0.397091
8,3.480486,9.323776,54.208165,0.645034
9,-1.405143,2.163494,51.626004,-1.569814


In [12]:
# Define the hyperparameter space
hyp_space  = [
    Integer(2, 10, name='num_layers'), # Num of layers in the network (depth)
    Integer(50, 200, name='num_units'), # Num of neurons in each hidden layer (width)
    Real(0.00001, 0.1, prior='log-uniform', name='learning_rate'), # Steps size at each iteration 
    Real(0.0, 0.5, name='dropout_rate'), # Probability of droping out a neuron
    Integer(20, 150, name='batch_size'), # Num of samples per batch
    Integer(50, 200, name='epochs') # Num of epochs (iterations over the entire dataset) during training
]

# Defining the Neural Network Model
def NN_model(num_layers, num_units, learning_rate, dropout_rate):
    inputs = Input(shape=(2,))
    x = Dense(num_units, activation='relu', kernel_regularizer='l2')(inputs)
    for _ in range(num_layers - 1):
        x = Dense(num_units, activation='relu', kernel_regularizer='l2')(x)
        x = Dropout(dropout_rate)(x)
    outputs = Dense(2, activation='linear')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    return model

# Define the objective function to minimize 
@use_named_args(hyp_space)
def objective(**params):
    num_layers = params['num_layers']
    num_units = params['num_units']
    learning_rate = params['learning_rate']
    dropout_rate = params['dropout_rate']
    batch_size = params['batch_size']
    epochs = params['epochs']

    model = NN_model(num_layers, num_units, learning_rate, dropout_rate)
    
    # Define K-fold cross-validation 
    # Splits data into multiple train and validation sets 
    kfold = KFold(n_splits=3)
    scores = []

    # Perform cross-validation 
    for train_idx, val_idx in kfold.split(df[['x', 'y']]):
        X_train, X_val = df[['x', 'y']].iloc[train_idx], df[['x', 'y']].iloc[val_idx]
        y_train, y_val = df[['z1', 'z2']].iloc[train_idx], df[['z1', 'z2']].iloc[val_idx]

        # Standardize the features
        scaler_x = StandardScaler().fit(X_train)
        X_train_scaled = scaler_x.transform(X_train)
        X_val_scaled = scaler_x.transform(X_val)

        # Train the model on scaled data
        model.fit(X_train_scaled, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

        # Evaluate the model
        score = model.evaluate(X_val_scaled, y_val, verbose=0)
        scores.append(score)
    
    # Returns mean of scores 
    return np.mean(scores)

In [13]:
# Perform Bayesian optimization
result = gp_minimize(objective, hyp_space, n_calls=50, random_state=0, acq_func='EI')

# Output best hyperparameters from BayesOpt
print("Best hyperparameters:")
print("num_layers:", result.x[0])
print("num_units:", result.x[1])
print("learning_rate:", result.x[2])
print("dropout_rate:", result.x[3])
print("batch_size:", result.x[4])
print("epochs:", result.x[5])

# Train the model with the best hyperparameters on the entire dataset
best_model = NN_model(
    num_layers=result.x[0],
    num_units=result.x[1],
    learning_rate=result.x[2],
    dropout_rate=result.x[3]
)

Best hyperparameters:
num_layers: 5
num_units: 200
learning_rate: 0.002052834479324138
dropout_rate: 0.5
batch_size: 20
epochs: 200


In [14]:
# Standardize the initial dataset
scaler_x = StandardScaler().fit(df[['x', 'y']])
X_scaled = scaler_x.transform(df[['x', 'y']])

# Train the best model on the full dataset
best_model.fit(X_scaled, df[['z1', 'z2']], epochs=result.x[5], batch_size=result.x[4], verbose=1)

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 3402.2886
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 3386.8394
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 3383.9602
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 3373.1094
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 3347.5801
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 3327.0806
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 3319.5103
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 3271.2737
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 3182.6279
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 3126.1736
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 3020.3291
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 2941.8533
Epoch 13/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 2704.0891
Epoch 14/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 2436.9719
Epoch 15/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms

In [61]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.problem import Problem
from pymoo.termination import get_termination
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.operators.crossover.sbx import SimulatedBinaryCrossover
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.optimize import minimize

# Define the objective functions based on the neural network outputs
# Problem is initialized with 2 decision vars and two objectives vars
class NNProblem(Problem):
    def __init__(self, model, scaler, maximize):
        super().__init__(n_var=2, n_obj=2, n_constr=0, xl=np.array([-15, 0]), xu=np.array([10, 15])) 
        self.model = model
        self.scaler = scaler
        self.maximize = maximize

    # Input data is scaled using a pre-fitted scaler
    # Predictions are made using NN model and stores them in an array
    def _evaluate(self, X, out, *args, **kwargs):
        X_scaled = self.scaler.transform(X)
        predictions = self.model.predict(X_scaled)
        # Adjust objectives based on whether they should be maximized or minimized
        for i in range(predictions.shape[1]):
            if self.maximize[i]:
                predictions[:, i] = -predictions[:, i]  # Convert maximization to minimization
        out["F"] = predictions

# Define the problem for NSGA-II (multi-objective evolutionary alorgirthm)
maximize = [False, False]
problem = NNProblem(best_model, scaler_x, maximize)

# Defining the NSGA-II algorithm
algorithm = NSGA2(
    pop_size=10, # Outputs 10 pareto optimal points
    sampling=FloatRandomSampling(),
    crossover=SimulatedBinaryCrossover(prob=0.9, eta=15),
    mutation=PolynomialMutation(eta=20), 
    eliminate_duplicates=True # Ensures duplicate solutions are disregarded
)

# Perform optimization
termination = get_termination("n_gen", 100) # Sets the termination criterion to stop after 100 generations
res = minimize(problem, algorithm, termination, seed=1, save_history=True, verbose=True)

# Extract Pareto-optimal solutions
pareto_set = res.X
pareto_front = res.F

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       10 |     10 |             - |             -
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
     2 |       20 |     10 |  0.0061874759 |         ideal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
     3 |       30 |     10 |  0.0059336667 |         ideal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
     4 |       40 |     10 |  0.0542370098 |         ideal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
     5 |       50 |     10 |  0.0178658652 |         ideal


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
    96 |      960 |     10 |  0.0265129969 |             f
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
    97 |      970 |     10 |  0.0235732534 |             f


c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
    98 |      980 |     10 |  0.0094006213 |             f
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
    99 |      990 |     10 |  0.0300651290 |             f


c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
   100 |     1000 |     10 |  0.0146859566 |             f


In [62]:
# Displaying results
print("Pareto-optimal solutions (inputs):")
print(pareto_set)
print("Pareto-optimal solutions (outputs):")
print(pareto_front)

# Choose the next samples based on Pareto-optimal solutions
next_samples = pd.DataFrame(pareto_set, columns=['x', 'y'])
next_samples["z1"] = pareto_front[:, 0]
next_samples["z2"] = pareto_front[:, 1]

Pareto-optimal solutions (inputs):
[[ 2.79164722e-01  5.32437199e+00]
 [-1.50000000e+01  1.18419280e-06]
 [-1.50000000e+01  1.18419280e-06]
 [-6.25177374e+00  5.83702385e+00]
 [-9.61064196e+00  4.72847610e+00]
 [-1.47149772e+01  4.79900880e+00]
 [-1.47149772e+01  7.95091401e-01]
 [-1.28654767e+01  5.31296874e+00]
 [-6.25177374e+00  2.90713093e-01]
 [-7.77692202e-01  1.18419280e-06]]
Pareto-optimal solutions (outputs):
[[  7.74339437   0.40654179]
 [371.50640869 -49.78916168]
 [371.50640869 -49.78916168]
 [ 91.50109863 -11.16776657]
 [178.09199524 -23.04954147]
 [279.28240967 -36.72463989]
 [351.58129883 -46.99617767]
 [232.94233704 -30.39001083]
 [187.75794983 -24.90224457]
 [ 44.49414062  -5.25044012]]


In [63]:
print("Next samples to evaluate:")
print(next_samples)

Next samples to evaluate:
           x         y          z1         z2
0   0.279165  5.324372    7.743394   0.406542
1 -15.000000  0.000001  371.506409 -49.789162
2 -15.000000  0.000001  371.506409 -49.789162
3  -6.251774  5.837024   91.501099 -11.167767
4  -9.610642  4.728476  178.091995 -23.049541
5 -14.714977  4.799009  279.282410 -36.724640
6 -14.714977  0.795091  351.581299 -46.996178
7 -12.865477  5.312969  232.942337 -30.390011
8  -6.251774  0.290713  187.757950 -24.902245
9  -0.777692  0.000001   44.494141  -5.250440


In [69]:
# Extract x and y columns from next_samples DataFrame
X_new = next_samples[['x', 'y']].values

Y_new = next_samples[['z1', 'z2']].values

# Scale the extracted data
X_new_scaled = scaler_x.transform(X_new)


c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [71]:
actual_values = branin(next_samples['x'], next_samples['y'])
actual_values2 = currin(next_samples['x'], next_samples['y'])

validated_values = pd.DataFrame(actual_values, columns=['z1'])
validated_values["z2"] = actual_values2

validated_values = pd.concat([next_samples[['x', 'y']], validated_values], axis=1)
validated_values

,x,y,z1,z2
0,0.279165,5.324372,19.288646,1.212398
1,-15.000000,0.000001,3476.600070,32.733381
2,-15.000000,0.000001,3476.600066,32.733381
3,-6.251774,5.837024,249.487187,8.393886
4,-9.610642,4.728476,812.778515,4.442503
5,-14.714977,4.799009,2770.778857,3.267299
6,-14.714977,0.795091,3207.966251,15.414481
7,-12.865477,5.312969,1829.317196,3.179779
8,-6.251774,0.290713,448.436383,83.936745
9,-0.777692,0.000001,70.363731,-5.508085


In [67]:
loss = best_model.evaluate(X_new_scaled, validated_values[['z1', 'z2']])

loss_values = []

# Store the loss value in the list
loss_values.append(loss)

# Convert the list to a DataFrame
loss_df = pd.DataFrame(loss_values, columns=['loss'])

# Print the DataFrame
print("Loss values stored in a DataFrame:")
print(loss_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 1836400.7500
Loss values stored in a DataFrame:
         loss
0  1836400.75
